In [10]:
import sys, os
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print("Added to path:", project_root)

Added to path: e:\implementing_cnn


In [15]:
import torch
import torch.nn.functional as F
from modules.convolution import Conv2DFunc

In [16]:
# ----- Test Forward Pass -----
B, C_in, H, W = 2, 3, 8, 8
C_out = 4
K = 3
stride = 1
padding = 1

# random input & kernel
x = torch.randn(B, C_in, H, W, dtype=torch.float64, requires_grad=True)
w = torch.randn(C_out, C_in, K, K, dtype=torch.float64, requires_grad=True)

# your function
y_custom = Conv2DFunc.apply(x, w, stride, padding)

# pytorch reference
y_ref = F.conv2d(x, w, stride=stride, padding=padding)

print("Output shape (custom):", y_custom.shape)
print("Output shape (PyTorch):", y_ref.shape)
print("Difference (L2):", (y_custom - y_ref).pow(2).sum().item())


Output shape (custom): torch.Size([2, 4, 8, 8])
Output shape (PyTorch): torch.Size([2, 4, 8, 8])
Difference (L2): 3.767301206700048e-28


In [17]:
# ----- Gradient Check -----

x = torch.randn(1, 3, 5, 5, dtype=torch.float64, requires_grad=True)
w = torch.randn(4, 3, 3, 3, dtype=torch.float64, requires_grad=True)

def func(x, w):
    return Conv2DFunc.apply(x, w, 1, 1)

# gradcheck requires double precision & small inputs
torch.autograd.gradcheck(func, (x, w))


True

In [18]:
y = Conv2DFunc.apply(x, w, 1, 1).sum()
y.backward()

print("Grad x mean:", x.grad.mean().item())
print("Grad w mean:", w.grad.mean().item())
print("Grad x shape:", x.grad.shape)
print("Grad w shape:", w.grad.shape)


Grad x mean: -1.0745762245330694
Grad w mean: 2.190133235965526
Grad x shape: torch.Size([1, 3, 5, 5])
Grad w shape: torch.Size([4, 3, 3, 3])
